In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import random
from datetime import datetime

# Set a fixed random seed for reproducibility
random.seed(9815)

# Ticker names
tickers = ["T02Y", "T03Y", "T05Y", "T07Y", "T10Y", "T20Y", "T30Y"]

def log_timestamp(message):
    """Add timestamp to logging messages"""
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    print(f"{current_time} {message}")

def decimal_to_fractional(integer_part, decimal_32, decimal_8):
    """Convert decimal price to fractional notation"""
    if decimal_8 == 4:
        decimal_8 = "+"
    return f"{integer_part}-{decimal_32:02d}{decimal_8}"

def price_generator():
    """Generate price data with proper spreads"""
    log_timestamp("Generating prices.txt")
    with open("prices.txt", "w") as f:
        price_num = 1000000
        for i in range(price_num):
            if i % 100000 == 0:
                log_timestamp(f"{i} prices generated")
            for ele in tickers:
                # Base price oscillation kept as is
                integer_ = random.randint(99, 101)
                decimal_32 = random.randint(10, 31)
                decimal_8 = random.randint(0, 3)
                
                # Implement proper spread oscillation between 1/128 and 1/64
                spread = 2 if i % 2 == 0 else 4  # Alternates between 1/128 (2 ticks) and 1/64 (4 ticks)
                
                bid_string = decimal_to_fractional(integer_, decimal_32, decimal_8)
                ask_decimal_8 = (decimal_8 + spread) % 8
                ask_decimal_32 = decimal_32 + (1 if decimal_8 + spread >= 8 else 0)
                ask_integer = integer_ + (1 if ask_decimal_32 >= 32 else 0)
                ask_decimal_32 = ask_decimal_32 % 32
                
                ask_string = decimal_to_fractional(ask_integer, ask_decimal_32, ask_decimal_8)
                
                f.write(f"{ele},{bid_string},{ask_string}\n")
    log_timestamp("Finished generating prices.txt")

def trade_generator():
    """Generate trade data with proper alternating patterns"""
    log_timestamp("Generating trades.txt")
    with open("trades.txt", "w") as f:
        books = ["TRSY1", "TRSY2", "TRSY3"]
        sides = ["BUY", "SELL"]
        trade_id = 0

        for ele in tickers:
            for i in range(10):
                trade_id += 1
                
                # Alternate between BUY and SELL
                current_side = sides[i % 2]
                
                # Cycle through quantities 1M to 5M
                quantity = (i % 5 + 1) * 1000000
                
                # Keep oscillation between 99-000 and 101-000 as specified
                price = "99-000" if current_side == "BUY" else "101-000"
                
                f.write(f"{ele},TradeId{trade_id},{books[i % 3]},{price},{quantity},{current_side}\n")
    log_timestamp("Finished generating trades.txt")

def market_data_generator():
    """Generate market data with proper size levels and spread behavior"""
    log_timestamp("Generating marketdata.txt")
    
    # Define size levels as per requirements
    SIZES = [10_000_000, 20_000_000, 30_000_000, 40_000_000, 50_000_000]
    
    # Define spread progression (1/128, 1/64, 3/128, 1/32) in ticks
    BASE_SPREADS = [2, 4, 6, 8]  # Each tick is 1/256th
    
    with open("marketdata.txt", "w") as f:
        number_orderbooks = 1000000
        for i in range(number_orderbooks):
            if i % 100000 == 0:
                log_timestamp(f"{i} orderbooks generated")
            
            for ele in tickers:
                # Base price setup (keeping existing oscillation)
                integer_part = random.randint(99, 101)
                decimal_32 = random.randint(12, 29)
                decimal_8 = random.randint(4, 7)
                
                # Get current base spread from the cycle
                base_spread = BASE_SPREADS[i % len(BASE_SPREADS)]
                
                s_to_file = ele
                for level in range(5):
                    # Increase spread for each level
                    level_spread = base_spread + (level * 2)
                    
                    # Calculate bid/ask prices
                    bid_decimal_8 = (decimal_8 - level_spread//2) % 8
                    ask_decimal_8 = (decimal_8 + level_spread//2) % 8
                    
                    # Handle decimal rollovers
                    bid_decimal_32 = decimal_32
                    ask_decimal_32 = decimal_32
                    bid_integer = integer_part
                    ask_integer = integer_part
                    
                    if decimal_8 - level_spread//2 < 0:
                        bid_decimal_32 -= 1
                        if bid_decimal_32 < 0:
                            bid_decimal_32 = 31
                            bid_integer -= 1
                    
                    if decimal_8 + level_spread//2 > 7:
                        ask_decimal_32 += 1
                        if ask_decimal_32 > 31:
                            ask_decimal_32 = 0
                            ask_integer += 1
                    
                    bid = decimal_to_fractional(bid_integer, bid_decimal_32, bid_decimal_8)
                    ask = decimal_to_fractional(ask_integer, ask_decimal_32, ask_decimal_8)
                    
                    s_to_file += f",{bid},{SIZES[level]},{ask},{SIZES[level]}"
                
                f.write(f"{s_to_file}\n")
    log_timestamp("Finished generating marketdata.txt")

def inquiries_generator():
    """Generate inquiry data with all required fields"""
    log_timestamp("Generating inquiries.txt")
    sides = ["BUY", "SELL"]
    
    with open("inquiries.txt", "w") as f:
        inquiry_id = 0
        for ele in tickers:
            for i in range(10):
                inquiry_id += 1
                
                # Alternate sides
                side = sides[i % 2]
                
                # Cycle through quantities like trades
                quantity = (i % 5 + 1) * 1000000
                
                # Include all required fields: ID, product, price, quantity, side, state
                f.write(f"{ele},INQUIRY_{inquiry_id:02d},100-000,{quantity},{side},RECEIVED\n")
                
    log_timestamp("Finished generating inquiries.txt")

if __name__ == "__main__":
    # Generate all data files
    price_generator()
    trade_generator()
    market_data_generator()
    inquiries_generator()

2024-12-22 22:06:38.968 Generating prices.txt
2024-12-22 22:06:38.973 0 prices generated
2024-12-22 22:06:40.065 100000 prices generated
2024-12-22 22:06:41.144 200000 prices generated
2024-12-22 22:06:42.233 300000 prices generated
2024-12-22 22:06:43.314 400000 prices generated
2024-12-22 22:06:44.396 500000 prices generated
2024-12-22 22:06:45.477 600000 prices generated
2024-12-22 22:06:46.564 700000 prices generated
2024-12-22 22:06:47.696 800000 prices generated
2024-12-22 22:06:48.796 900000 prices generated
2024-12-22 22:06:49.880 Finished generating prices.txt
2024-12-22 22:06:49.881 Generating trades.txt
2024-12-22 22:06:49.881 Finished generating trades.txt
2024-12-22 22:06:49.881 Generating marketdata.txt
2024-12-22 22:06:49.918 0 orderbooks generated
2024-12-22 22:06:53.841 100000 orderbooks generated
2024-12-22 22:06:57.816 200000 orderbooks generated
2024-12-22 22:07:01.779 300000 orderbooks generated
2024-12-22 22:07:05.680 400000 orderbooks generated
2024-12-22 22:07:0